In [2]:
import pandas as pd; 
pd.set_option('display.max_rows', None)

index = ['QB', 'RB', 'WR']

df = pd.DataFrame({
    'A': [20, 19, 15],
    'B': [25, 11, 14]
}, index=index)

df.head()

,A,B
QB,20,25
RB,19,11
WR,15,14


In [8]:
df.sum()

A    54
B    50
dtype: int64

In [3]:
df['A - B'] = df['A'] - df['B']
df.head()

,A,B,A - B
QB,20,25,-5
RB,19,11,8
WR,15,14,1


In [13]:
df.max(axis=1)

QB    25
RB    19
WR    15
dtype: int64

In [4]:
df['VOR'] = abs(df['A - B'])
df.sort_values(by='VOR', ascending=False)

,A,B,A - B,VOR
RB,19,11,8,8
QB,20,25,-5,5
WR,15,14,1,1


In [5]:
from bs4 import BeautifulSoup as bs
import requests
url = "https://www.fantasypros.com/nfl/adp/ppr-overall.php"

def make_adp_df():
    res = requests.get(url)
    if res.ok:
        soup = bs(res.content, 'html.parser')
        table = soup.find('table', {'id': 'data'})
        df = pd.read_html(str(table))[0]
        print('output after reading in: \n\n', df.head(), '\n')
        df= df[['Player Team (Bye)', 'POS', 'AVG']]
        print('Output after filtering:\n\n', df.head(), '\n')
        df['PLAYER'] = df['Player Team (Bye)'].apply(lambda x: ' '.join(x.split()[:-2])) # removing the team and position
        df['POS'] = df['POS'].apply(lambda x: x[:2]) # removing the position rank
        
        df = df[['PLAYER', 'POS', 'AVG']].sort_values(by='AVG')
        
        print('Final output: \n\n', df.head())
        return df
        
    else:
        print('oops, something didn\'t work right', res.status_code)
        
df = make_adp_df()
    


output after reading in: 

    Rank             Player Team (Bye)  POS  ESPN  MFL  FFC  RTSports  NFL  \
0     1  Christian McCaffrey CAR (13)  RB1   1.0  NaN  1.0       1.0  NaN   
1     2       Saquon Barkley NYG (11)  RB2   2.0  NaN  2.0       2.0  NaN   
2     3         Michael Thomas NO (6)  WR1   4.0  NaN  4.0       5.0  NaN   
3     4      Ezekiel Elliott DAL (10)  RB3   3.0  NaN  7.0       3.0  NaN   
4     5           Dalvin Cook MIN (7)  RB4   6.0  NaN  3.0       6.0  NaN   

   Fantrax  AVG  
0      1.0  1.0  
1      2.0  2.0  
2      3.0  4.0  
3      4.0  4.3  
4      5.0  5.0   

Output after filtering:

               Player Team (Bye)  POS  AVG
0  Christian McCaffrey CAR (13)  RB1  1.0
1       Saquon Barkley NYG (11)  RB2  2.0
2         Michael Thomas NO (6)  WR1  4.0
3      Ezekiel Elliott DAL (10)  RB3  4.3
4           Dalvin Cook MIN (7)  RB4  5.0 

Final output: 

                 PLAYER POS  AVG
0  Christian McCaffrey  RB  1.0
1       Saquon Barkley  RB  2.0
2     

In [6]:
replacement_players = {
    'RB': '',
    'WR': '',
    'TE': '',
    'QB': ''
}

for _, row in df[:100].iterrows():
    position = row['POS']
    player = row['PLAYER']
    replacement_players[position] = player
    
replacement_players

{'RB': 'Tevin Coleman',
 'WR': 'Diontae Johnson',
 'TE': 'Jared Cook',
 'QB': 'Carson Wentz'}

In [10]:
# each position has a different associated URL. We'll create a string format here and loop through the possible positions
BASE_URL = 'https://www.fantasypros.com/nfl/projections/{position}.php?week=draft'

def make_projection_df():
    
    # we are going to concatenate our individual position dfs into this larger final_df
    final_df = pd.DataFrame()
    
    #url has positions in lower case
    for position in ['rb', 'qb', 'te', 'wr']:
        
        res = requests.get(BASE_URL.format(position=position)) # format our url with the position
        if res.ok:
            soup = bs(res.content, 'html.parser')
            table = soup.find('table', {'id': 'data'})
            df = pd.read_html(str(table))[0]
            
            df.columns = df.columns.droplevel(level=0) # our data has a multi-level column index. The first column level is useless so let's drop it.
            df['PLAYER'] = df['Player'].apply(lambda x: ' '.join(x.split()[:-1])) # fixing player name to not include team
            # if you're not doing PPR, don't include this. If you're doing Half PPR,
            # multiply receptions * 1/2
            if 'REC' in df.columns:
                df['FPTS'] = df['FPTS'] + df['REC'] # add receptions if they're in there. 
            
            df['POS'] = position.upper() # add a position column
            
            df = df[['PLAYER', 'POS', 'FPTS']]
            final_df = pd.concat([final_df, df]) # iteratively add to our final_df
        else:
            print('oops something didn\'t work right', res.status_code)
            return
    
    final_df = final_df.sort_values(by='FPTS', ascending=False) # sort df in descending order on FPTS column
    
    return final_df
            
df = make_projection_df()
df.head()

ProxyError: HTTPSConnectionPool(host='www.fantasypros.com', port=443): Max retries exceeded with url: /nfl/projections/rb.php?week=draft (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 403 URLBlocked')))

In [33]:
replacement_values = {
    'RB': 0,
    'WR': 0,
    'QB': 0,
    'TE': 0
}

for position, player in replacement_players.items():
    replacement_values[position] = df.loc[df['PLAYER'] == player].values[0, -1]
    
replacement_values

{'RB': 132.4, 'WR': 178.10000000000002, 'QB': 269.9, 'TE': 150.5}

In [34]:
df['VOR'] = df.apply(
    lambda row: row['FPTS'] - replacement_values.get(row['POS']), axis=1
)
df.head()

,PLAYER,POS,FPTS,VOR
0,Christian McCaffrey,RB,371.6,239.2
0,Lamar Jackson,QB,352.5,82.6
1,Patrick Mahomes,QB,340.0,70.1
0,Michael Thomas,WR,326.6,148.5
1,Saquon Barkley,RB,316.9,184.5


In [38]:
df = df.sort_values(by='VOR', ascending=False)
df['VALUERANK'] = df['VOR'].rank(ascending=False)
df.to_csv('VAB.csv')
df.head(150)


,PLAYER,POS,FPTS,VOR,VALUERANK
0,Christian McCaffrey,RB,371.6,239.2,1.0
1,Saquon Barkley,RB,316.9,184.5,2.0
2,Ezekiel Elliott,RB,308.6,176.2,3.0
4,Dalvin Cook,RB,298.5,166.1,4.0
5,Alvin Kamara,RB,293.6,161.2,5.0
0,Michael Thomas,WR,326.6,148.5,6.0
3,Derrick Henry,RB,260.2,127.8,7.0
6,Clyde Edwards-Helaire,RB,254.9,122.5,8.0
12,Kenyan Drake,RB,248.8,116.4,9.0
11,Miles Sanders,RB,246.3,113.9,10.0
